Строим запрос к нашему сервису предсказания вероятности покупки страховки.

In [30]:
import requests
import numpy as np
import pandas as pd
import urllib.request
import json

In [9]:
X_test = pd.read_csv("X_test.csv")
y_test = pd.read_csv("y_test.csv")

In [18]:
# функция формирования запроса
def send_json(x):
    d1 = int(x['Age'])
    d2 = int(x['AnnualIncome'])
    d3 = int(x['FamilyMembers'])
    d4 = int(x['ChronicDiseases'])
    d5 = str(x['FrequentFlyer'])
    d6  = str(x['EverTravelledAbroad'])
    body = {
        'Age': d1, 
        'AnnualIncome': d2,
        'FamilyMembers': d3,
        'ChronicDiseases': d4,
        'FrequentFlyer': d5,
        'EverTravelledAbroad': d6
        }
    myurl = 'http://127.0.0.1:5000/predict'
    headers = {'content-type': 'application/json; charset=utf-8'}
    response = requests.post(myurl, json=body, headers=headers)
    return response.json()['predictions']

Проверка на одиночном примере:

In [16]:
a = {'Age':28, 'AnnualIncome':258000, 'FamilyMembers':3, 'ChronicDiseases':0, 'FrequentFlyer':'Yes', 'EverTravelledAbroad':'No'}

In [23]:
response = send_json(a)
response

0.095

Сделаем N запросов и оценим время.

In [37]:
N = 100

In [38]:
%%time
predictions = X_test[
                     ['Age', 'AnnualIncome', 'FamilyMembers', 'ChronicDiseases', 'FrequentFlyer', 'EverTravelledAbroad']
                     ].iloc[:N].apply(lambda x: send_json(x), axis=1)

Wall time: 3.9 s


In [29]:
predictions.values[:10]

array([0.33466667, 0.21560498, 0.57480952, 0.        , 0.575     ,
       0.        , 0.45759524, 0.37544048, 1.        , 0.45      ])

Посчитаем метрику:

In [31]:
from sklearn.metrics import roc_auc_score, roc_curve, precision_recall_curve
from sklearn.metrics import f1_score

In [32]:
precision, recall, thresholds = precision_recall_curve(y_test[:N], predictions)

fscore = (2 * precision * recall) / (precision + recall)
# locate the index of the largest f score
ix = np.argmax(fscore)
print(f'Best Threshold={thresholds[ix]}, F-Score={fscore[ix]:.3f}, Precision={precision[ix]:.3f}, Recall={recall[ix]:.3f}')

Best Threshold=0.4575952380952381, F-Score=0.750, Precision=0.750, Recall=0.750


Метрики качества прогнозирования на 100 тестовых объектах близки к показателям на всей тестовой выоборке. Модель работает.